In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/next-word-prediction/1661-0.txt


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer 


2025-04-19 11:23:31.269852: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745061811.516454      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745061811.593187      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
with open('/kaggle/input/next-word-prediction/1661-0.txt', 'r', encoding='utf-8') as file:
    text = file.read()

print(text[:1000])  # print first 1000 characters to check


﻿
Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.net


Title: The Adventures of Sherlock Holmes

Author: Arthur Conan Doyle

Release Date: November 29, 2002 [EBook #1661]
Last Updated: May 20, 2019

Language: English

Character set encoding: UTF-8

*** START OF THIS PROJECT GUTENBERG EBOOK THE ADVENTURES OF SHERLOCK HOLMES ***



Produced by an anonymous Project Gutenberg volunteer and Jose Menendez



cover



The Adventures of Sherlock Holmes



by Arthur Conan Doyle



Contents


   I.     A Scandal in Bohemia
   II.    The Red-Headed League
   III.   A Case of Identity
   IV.    The Boscombe Valley Mystery
   V.     The Five Orange Pips
   VI.    The Man with the Twisted Lip
   VII.   The Adventure of the Blue 

In [4]:
tokenizer=Tokenizer()

In [5]:
tokenizer.fit_on_texts([text])

In [6]:
len(tokenizer.word_index)
tokenizer.word_index.items()

dict_items([('the', 1), ('and', 2), ('to', 3), ('of', 4), ('a', 5), ('i', 6), ('”', 7), ('in', 8), ('that', 9), ('it', 10), ('he', 11), ('was', 12), ('you', 13), ('his', 14), ('is', 15), ('my', 16), ('have', 17), ('with', 18), ('as', 19), ('had', 20), ('at', 21), ('which', 22), ('for', 23), ('be', 24), ('not', 25), ('me', 26), ('but', 27), ('from', 28), ('we', 29), ('this', 30), ('said', 31), ('upon', 32), ('there', 33), ('holmes', 34), ('him', 35), ('so', 36), ('her', 37), ('she', 38), ('all', 39), ('’', 40), ('been', 41), ('your', 42), ('on', 43), ('very', 44), ('by', 45), ('one', 46), ('are', 47), ('“i', 48), ('were', 49), ('an', 50), ('no', 51), ('would', 52), ('out', 53), ('what', 54), ('then', 55), ('up', 56), ('when', 57), ('man', 58), ('could', 59), ('has', 60), ('do', 61), ('into', 62), ('or', 63), ('little', 64), ('will', 65), ('who', 66), ('mr', 67), ('if', 68), ('some', 69), ('down', 70), ('see', 71), ('now', 72), ('our', 73), ('should', 74), ('may', 75), ('am', 76), ('us',

In [7]:
# Get the first 1000 items of the tokenizer.word_index
top_1000 = list(tokenizer.word_index.items())[:1000]
top_1000


[('the', 1),
 ('and', 2),
 ('to', 3),
 ('of', 4),
 ('a', 5),
 ('i', 6),
 ('”', 7),
 ('in', 8),
 ('that', 9),
 ('it', 10),
 ('he', 11),
 ('was', 12),
 ('you', 13),
 ('his', 14),
 ('is', 15),
 ('my', 16),
 ('have', 17),
 ('with', 18),
 ('as', 19),
 ('had', 20),
 ('at', 21),
 ('which', 22),
 ('for', 23),
 ('be', 24),
 ('not', 25),
 ('me', 26),
 ('but', 27),
 ('from', 28),
 ('we', 29),
 ('this', 30),
 ('said', 31),
 ('upon', 32),
 ('there', 33),
 ('holmes', 34),
 ('him', 35),
 ('so', 36),
 ('her', 37),
 ('she', 38),
 ('all', 39),
 ('’', 40),
 ('been', 41),
 ('your', 42),
 ('on', 43),
 ('very', 44),
 ('by', 45),
 ('one', 46),
 ('are', 47),
 ('“i', 48),
 ('were', 49),
 ('an', 50),
 ('no', 51),
 ('would', 52),
 ('out', 53),
 ('what', 54),
 ('then', 55),
 ('up', 56),
 ('when', 57),
 ('man', 58),
 ('could', 59),
 ('has', 60),
 ('do', 61),
 ('into', 62),
 ('or', 63),
 ('little', 64),
 ('will', 65),
 ('who', 66),
 ('mr', 67),
 ('if', 68),
 ('some', 69),
 ('down', 70),
 ('see', 71),
 ('now', 72),


In [8]:
# for sentence in text.split('\n'):
#     print(tokenizer.texts_to_sequences([sentence])[0])


clean_sequences = []
input_sequence=[]
for sentence in text.split('\n'):
    sequence = tokenizer.texts_to_sequences([sentence])[0]
    if sequence:  # only add non-empty sequences
        clean_sequences.append(sequence)
        tokenized_sentence=tokenizer.texts_to_sequences([sentence])[0]
        for i in range(1,len(tokenized_sentence)):
            input_sequence.append(tokenized_sentence[:i+1])
        


In [9]:
input_sequence

[[145, 4790],
 [145, 4790, 1],
 [145, 4790, 1, 1020],
 [145, 4790, 1, 1020, 4],
 [145, 4790, 1, 1020, 4, 128],
 [145, 4790, 1, 1020, 4, 128, 34],
 [145, 4790, 1, 1020, 4, 128, 34, 45],
 [145, 4790, 1, 1020, 4, 128, 34, 45, 611],
 [145, 4790, 1, 1020, 4, 128, 34, 45, 611, 2235],
 [145, 4790, 1, 1020, 4, 128, 34, 45, 611, 2235, 2236],
 [30, 1021],
 [30, 1021, 15],
 [30, 1021, 15, 23],
 [30, 1021, 15, 23, 1],
 [30, 1021, 15, 23, 1, 275],
 [30, 1021, 15, 23, 1, 275, 4],
 [30, 1021, 15, 23, 1, 275, 4, 394],
 [30, 1021, 15, 23, 1, 275, 4, 394, 2237],
 [30, 1021, 15, 23, 1, 275, 4, 394, 2237, 21],
 [30, 1021, 15, 23, 1, 275, 4, 394, 2237, 21, 51],
 [30, 1021, 15, 23, 1, 275, 4, 394, 2237, 21, 51, 1676],
 [30, 1021, 15, 23, 1, 275, 4, 394, 2237, 21, 51, 1676, 2],
 [30, 1021, 15, 23, 1, 275, 4, 394, 2237, 21, 51, 1676, 2, 18],
 [572, 51],
 [572, 51, 3398],
 [572, 51, 3398, 3399],
 [572, 51, 3398, 3399, 13],
 [572, 51, 3398, 3399, 13, 75],
 [572, 51, 3398, 3399, 13, 75, 817],
 [572, 51, 3398, 33

In [10]:
max_len=max([len(x) for x in input_sequence])
from tensorflow.keras.preprocessing.sequence import pad_sequences
pad_input_sequence=pad_sequences(input_sequence,maxlen=max_len,padding='pre')

In [11]:
pad_input_sequence

array([[   0,    0,    0, ...,    0,  145, 4790],
       [   0,    0,    0, ...,  145, 4790,    1],
       [   0,    0,    0, ..., 4790,    1, 1020],
       ...,
       [   0,    0,    0, ...,    3,  360,   83],
       [   0,    0,    0, ...,  360,   83,  358],
       [   0,    0,    0, ...,   83,  358, 1673]], dtype=int32)

In [12]:
x=pad_input_sequence[:,:-1]
x.shape

(101619, 19)

In [13]:
y=pad_input_sequence[:,-1]
y.shape

(101619,)

In [14]:
vocab_size = len(tokenizer.word_index) + 1  # +1 for padding (0)
vocab_size



8932

In [15]:
from tensorflow.keras.utils import to_categorical
y=to_categorical(y,num_classes=8932)
y.shape

(101619, 8932)

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Embedding,Dense

In [17]:
model=Sequential()
model.add(Embedding(8932,100,input_length=19))
model.add(LSTM(1000))
model.add(Dense(8932,activation='softmax'))
model.build(input_shape=(None, 19))



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1745061826.800186      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1745061826.800903      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [18]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [19]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 19, 100)             │         893,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 1000)                │       4,404,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 8932)                │       8,940,932 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,238,132 (54.31 MB)

 Trainable params: 14,238,132 (54.31 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
model.fit(x,y,epochs=100)

Epoch 1/100


I0000 00:00:1745061844.483527      64 cuda_dnn.cc:529] Loaded cuDNN version 90300


3176/3176 ━━━━━━━━━━━━━━━━━━━━ 64s 19ms/step - accuracy: 0.0693 - loss: 6.4274
Epoch 2/100
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 63s 20ms/step - accuracy: 0.1365 - loss: 5.3081
Epoch 3/100
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 62s 20ms/step - accuracy: 0.1671 - loss: 4.6844
Epoch 4/100
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 62s 20ms/step - accuracy: 0.2173 - loss: 3.9158
Epoch 5/100
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 63s 20ms/step - accuracy: 0.3468 - loss: 2.9797
Epoch 6/100
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 63s 20ms/step - accuracy: 0.5139 - loss: 2.1462
Epoch 7/100
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 63s 20ms/step - accuracy: 0.6600 - loss: 1.5197
Epoch 8/100
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 62s 20ms/step - accuracy: 0.7553 - loss: 1.1104
Epoch 9/100
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 63s 20ms/step - accuracy: 0.8097 - loss: 0.8663
Epoch 10/100
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 63s 20ms/step - accuracy: 0.8365 - loss: 0.7385
Epoch 11/100
3176/3176 ━━━━━━━━━━━━━━━━━━━━ 63s 20ms/step - accuracy: 0.8572 - loss: 0.6452
Epoch 12/

In [21]:
textd="mail"
# tokeinezed
token_text = tokenizer.texts_to_sequences([textd])[0]
# padding
padded_text = pad_sequences([token_text], maxlen=20, padding='pre')

print(padded_text)
for word,index in tokenizer.word_index.item():
    if index==poc:
        print(word)

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0 3164]]


AttributeError: 'dict' object has no attribute 'item'

In [ ]:
max_len

In [ ]:
np.argmax(model.predict(padded_text))

In [ ]:
tokenizer.word_index